# Exercise 5 Solutions
This notebook contains solutions to the week 5 exercises.

## Exercise 5.1
Compute the projection matrices and image projections of $Q$.

In [1]:

import numpy as np
# Camera intrinsics
K = np.array([[700,0,600],
              [0,700,400],
              [0,0,1]])
# Rotations and translations
R1 = np.eye(3)
R2 = np.eye(3)
t1 = np.array([[0],[0],[1]])
t2 = np.array([[0],[0],[20]])
# Projection matrices
P1 = K @ np.hstack([R1, t1])
P2 = K @ np.hstack([R2, t2])
# 3D point
Q = np.array([1,1,0])
Qh = np.hstack([Q,1])

def project(P, Qh):
    q = P @ Qh
    return (q/q[2])[:2]

q1 = project(P1, Qh)
q2 = project(P2, Qh)
P1, P2, q1, q2


(array([[700.,   0., 600., 600.],
        [  0., 700., 400., 400.],
        [  0.,   0.,   1.,   1.]]),
 array([[7.0e+02, 0.0e+00, 6.0e+02, 1.2e+04],
        [0.0e+00, 7.0e+02, 4.0e+02, 8.0e+03],
        [0.0e+00, 0.0e+00, 1.0e+00, 2.0e+01]]),
 array([1300., 1100.]),
 array([635., 435.]))

## Exercise 5.2
Triangulate from noisy observations and compute reprojection error.

In [2]:

# Add noise to projections
q1_tilde = q1 + np.array([1,-1])
q2_tilde = q2 + np.array([1,-1])

# Linear triangulation function

def triangulate(qs, Ps):
    A = []
    for q, P in zip(qs, Ps):
        x, y = q
        A.append(x*P[2] - P[0])
        A.append(y*P[2] - P[1])
    A = np.array(A)
    _, _, Vt = np.linalg.svd(A)
    X = Vt[-1]
    X = X/X[3]
    return X[:3]

Q_tilde = triangulate([q1_tilde, q2_tilde], [P1, P2])

# Reproject
q1_hat = project(P1, np.hstack([Q_tilde,1]))
q2_hat = project(P2, np.hstack([Q_tilde,1]))

# Errors
err1 = np.linalg.norm(q1_hat - q1_tilde)
err2 = np.linalg.norm(q2_hat - q2_tilde)

# Distance to true Q
err_Q = np.linalg.norm(Q_tilde - Q)
Q_tilde, err1, err2, err_Q


(array([1.01527507e+00, 9.85270570e-01, 2.85786810e-04]),
 np.float64(13.4330189881917),
 np.float64(0.6717725840473774),
 np.float64(0.021221817353380742))

## Exercise 5.3
Implement nonlinear triangulation using least squares.

In [3]:

from scipy.optimize import least_squares

def triangulate_nonlin(qs, Ps):
    def compute_residuals(X):
        Xh = np.hstack([X,1])
        res = []
        for q, P in zip(qs, Ps):
            q_hat = project(P, Xh)
            res.extend(q_hat - q)
        return np.array(res)
    x0 = triangulate(qs, Ps)
    res = least_squares(compute_residuals, x0)
    return res.x


## Exercise 5.4
Use the nonlinear triangulation and evaluate reprojection error.

In [4]:

Q_hat = triangulate_nonlin([q1_tilde, q2_tilde], [P1, P2])

q1_hat_nl = project(P1, np.hstack([Q_hat,1]))
q2_hat_nl = project(P2, np.hstack([Q_hat,1]))
err1_nl = np.linalg.norm(q1_hat_nl - q1_tilde)
err2_nl = np.linalg.norm(q2_hat_nl - q2_tilde)
err_Q_nl = np.linalg.norm(Q_hat - Q)
Q_hat, err1_nl, err2_nl, err_Q_nl


(array([1.00153898e+00, 9.98546325e-01, 4.27519045e-05]),
 np.float64(0.06701026911364885),
 np.float64(1.34015086688393),
 np.float64(0.002117415491010624))

## Exercises 5.5 - 5.11
The following exercises involve camera calibration using real images.
Code templates are provided for guidance. Replace placeholders with your own images.

### Exercise 5.5
Capture approximately twenty images of a checkerboard pattern from different viewpoints.

### Exercise 5.6
Load images and ensure consistent orientation.

In [ ]:

import cv2
import glob
image_paths = sorted(glob.glob('calibration_images/*.jpg'))
images = [cv2.imread(p) for p in image_paths if cv2.imread(p) is not None]
if not images:
    print('No images found. Place calibration images in calibration_images/*.jpg')
else:
    sizes = [im.shape for im in images]
    if len(set(sizes)) != 1:
        print('Warning: images have inconsistent sizes.')


### Exercise 5.7
Detect checkerboard corners in all images.

In [ ]:

pattern_size = (9,6)
objpoints = []
imgpoints = []
objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:pattern_size[0],0:pattern_size[1]].T.reshape(-1,2)
for im in images:
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, pattern_size)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)


### Exercise 5.8
Calibrate the camera using detected corners.

In [ ]:

if objpoints and imgpoints:
    ret, K_est, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, images[0].shape[1::-1], None, None,
        flags=cv2.CALIB_FIX_K1+cv2.CALIB_FIX_K2+cv2.CALIB_FIX_K3+cv2.CALIB_FIX_K4+              cv2.CALIB_FIX_K5+cv2.CALIB_FIX_K6+cv2.CALIB_ZERO_TANGENT_DIST)
    print('Estimated K:', K_est)
else:
    print('Calibration requires detected corners.')


### Exercise 5.9
Reproject checkerboard corners and compute RMSE per frame.

In [ ]:

if objpoints and imgpoints:
    rmse = []
    for objp,corners,rvec,tvec in zip(objpoints,imgpoints,rvecs,tvecs):
        proj,_ = cv2.projectPoints(objp, rvec, tvec, K_est, dist)
        err = np.linalg.norm(proj.reshape(-1,2) - corners.reshape(-1,2), axis=1)
        rmse.append(np.sqrt((err**2).mean()))
    print('Max RMSE:', np.max(rmse))
else:
    print('Need calibration results to compute RMSE.')


### Exercise 5.10
Project a 3D box into one of the images.

In [ ]:

# Placeholder for projecting a 3D box; assumes box3d() from week 1
try:
    from ex01 import box3d
    if objpoints and imgpoints:
        Q_box = 2*box3d() + 1
        rvec, tvec = rvecs[0], tvecs[0]
        proj,_ = cv2.projectPoints(Q_box, rvec, tvec, K_est, dist)
        print('Projected box points shape:', proj.shape)
    else:
        print('Calibration required before projection.')
except Exception as e:
    print('box3d function not available:', e)


### Exercise 5.11
Repeat calibration allowing first order distortion.

In [ ]:

if objpoints and imgpoints:
    ret2, K2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(objpoints, imgpoints, images[0].shape[1::-1], None, None)
    print('Reprojection error with distortion:', ret2)
else:
    print('Calibration requires detected corners.')
